In [9]:
from tensorflow.keras.layers import Dense, Flatten, Dropout,Input
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten,MaxPooling2D,AveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adam,schedules
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, History
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import confusion_matrix
import random
import numpy as np
import os,cv2
import matplotlib
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import io

TRAIN_SPLIT = 0.85
DIR = './Main Dataset/'
imgH = 32
imgW = 32
index_of_image = []
comapre_to_test = []


In [10]:
def prepare_image_array(imgDir, imgW, imgH):
    imgList = os.listdir(imgDir)
    # print(imgList)
    n = len(imgList)

    imgSet = []
    for i in range(n):
        imgPath = imgDir + imgList[i]
        if (os.path.exists(imgPath)):
            img = cv2.imread(imgPath)
            index_of_image.append(imgList[i])
            resizedImg = cv2.resize(img, (imgW, imgH))
            rgbImg = cv2.cvtColor(resizedImg, cv2.COLOR_BGR2RGB)
            imgSet.append(rgbImg)
        else:
            print("It is not a valid image path.")

    print("total image "+str(len(imgSet)))
    imgSet = np.array(imgSet, dtype=np.uint8)
    print("total shape "+str(imgSet.shape))

    return imgSet

In [11]:

print("running")
imgDir = DIR + 'Normal/'
imgSet1 = prepare_image_array(imgDir, imgW, imgH)
m = imgSet1.shape[0]
print(m)

imgDir = DIR + 'OverExposed/'
imgSet2 = prepare_image_array(imgDir, imgW, imgH)
n = imgSet2.shape[0]

imgDir = DIR + 'UnderExposed/'
imgSet3 = prepare_image_array(imgDir, imgW, imgH)
o = imgSet3.shape[0]


# Put all image data into one array.
imgSet = np.concatenate((imgSet1, imgSet2, imgSet3), axis=0)
print(imgSet.shape)

labelSet1 = [0] * m
# print(labelSet1)
labelSet2 = [1] * n 
# print(labelSet2)
labelSet3 = [2] * o
# print(labelSet3)

labelSet = np.concatenate((labelSet1, labelSet2, labelSet3), axis=0)


p = imgSet.shape[0]  # p = n + m + o
indices = np.arange(p)
random.shuffle(indices)
index_of_image = np.array(index_of_image)
index_of_image = index_of_image[indices]
imgSet = imgSet[indices]
labelSet = labelSet[indices]
# print(labelSet[:20])
labelSet = labelSet.reshape(p, 1)

#   one hot encoding

labelSet = to_categorical(labelSet)

# print(labelSet[:20])

# print(imgSet[0])
imgSet = imgSet.astype(np.float32)
imgSet = imgSet/255.0

r = int(p * TRAIN_SPLIT)
trainX = imgSet[:r]
trainY = labelSet[:r]
testX = imgSet[r:]
comapre_to_test = index_of_image[r:]
testY = labelSet[r:]
print(len(index_of_image))


running
total image 3001
total shape (3001, 32, 32, 3)
3001
total image 3004
total shape (3004, 32, 32, 3)
total image 2979
total shape (2979, 32, 32, 3)
(8984, 32, 32, 3)
8984


In [12]:

inputs = Input(shape=(imgH, imgW, 3))
x = Flatten()(inputs)
x = Dense(units=1024, activation="relu")(x)
x = Dense(units=512, activation="relu")(x)
x = Dense(units=256, activation="relu")(x)
x = Dropout(.3)(x)
x = Dense(units=128, activation="relu")(x)
x = Dense(units=64, activation="relu")(x)
x = Dense(units=32, activation="relu")(x)
x = Dense(units=8, activation="relu")(x)
outputs = Dense(units=3, activation='softmax')(x)

model = Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 1024)              3146752   
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               32896 

In [13]:
output = []
history_dic = []
lr_schedule = schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=10000,
    decay_rate=0.9)



  # callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)

sgd = SGD(learning_rate=lr_schedule)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
callbackList = [EarlyStopping(monitor='val_loss', patience=50), History()]
history = model.fit(trainX, trainY, epochs=300, batch_size=128, validation_split=0.2, callbacks=callbackList)
loss, acc = model.evaluate(testX, testY)
print("\n\naccuracy  = %s"%( str(acc)))


Epoch 1/300
48/48 [==============================] - 1s 20ms/step - loss: 1.0494 - accuracy: 0.3420 - val_loss: 1.0401 - val_accuracy: 0.3253
Epoch 2/300
48/48 [==============================] - 1s 17ms/step - loss: 1.0317 - accuracy: 0.3484 - val_loss: 1.0252 - val_accuracy: 0.3292
Epoch 3/300
48/48 [==============================] - 1s 17ms/step - loss: 1.0200 - accuracy: 0.3525 - val_loss: 1.0120 - val_accuracy: 0.3508
Epoch 4/300
48/48 [==============================] - 1s 17ms/step - loss: 1.0079 - accuracy: 0.3679 - val_loss: 1.0008 - val_accuracy: 0.3757
Epoch 5/300
48/48 [==============================] - 1s 17ms/step - loss: 0.9973 - accuracy: 0.3892 - val_loss: 0.9903 - val_accuracy: 0.4051
Epoch 6/300
48/48 [==============================] - 1s 17ms/step - loss: 0.9885 - accuracy: 0.4049 - val_loss: 0.9797 - val_accuracy: 0.4346
Epoch 7/300
48/48 [==============================] - 1s 17ms/step - loss: 0.9795 - accuracy: 0.4386 - val_loss: 0.9707 - val_accuracy: 0.4588
Epoch 

48/48 [==============================] - 1s 17ms/step - loss: 0.7083 - accuracy: 0.7051 - val_loss: 0.6950 - val_accuracy: 0.7042
Epoch 59/300
48/48 [==============================] - 1s 17ms/step - loss: 0.7042 - accuracy: 0.7055 - val_loss: 0.6929 - val_accuracy: 0.7088
Epoch 60/300
48/48 [==============================] - 1s 18ms/step - loss: 0.7019 - accuracy: 0.7094 - val_loss: 0.6887 - val_accuracy: 0.7134
Epoch 61/300
48/48 [==============================] - 1s 16ms/step - loss: 0.6981 - accuracy: 0.7123 - val_loss: 0.6876 - val_accuracy: 0.7088
Epoch 62/300
48/48 [==============================] - 1s 15ms/step - loss: 0.6940 - accuracy: 0.7140 - val_loss: 0.6838 - val_accuracy: 0.7147
Epoch 63/300
48/48 [==============================] - 1s 17ms/step - loss: 0.6935 - accuracy: 0.7120 - val_loss: 0.6801 - val_accuracy: 0.7153
Epoch 64/300
48/48 [==============================] - 1s 16ms/step - loss: 0.6953 - accuracy: 0.7101 - val_loss: 0.6789 - val_accuracy: 0.7199
Epoch 65/300

48/48 [==============================] - 1s 16ms/step - loss: 0.5800 - accuracy: 0.7716 - val_loss: 0.5959 - val_accuracy: 0.7579
Epoch 116/300
48/48 [==============================] - 1s 17ms/step - loss: 0.5814 - accuracy: 0.7670 - val_loss: 0.5978 - val_accuracy: 0.7526
Epoch 117/300
48/48 [==============================] - 1s 17ms/step - loss: 0.5819 - accuracy: 0.7688 - val_loss: 0.5999 - val_accuracy: 0.7520
Epoch 118/300
48/48 [==============================] - 1s 17ms/step - loss: 0.5831 - accuracy: 0.7711 - val_loss: 0.5936 - val_accuracy: 0.7618
Epoch 119/300
48/48 [==============================] - 1s 17ms/step - loss: 0.5749 - accuracy: 0.7718 - val_loss: 0.5946 - val_accuracy: 0.7572
Epoch 120/300
48/48 [==============================] - 1s 16ms/step - loss: 0.5756 - accuracy: 0.7710 - val_loss: 0.5925 - val_accuracy: 0.7572
Epoch 121/300
48/48 [==============================] - 1s 16ms/step - loss: 0.5764 - accuracy: 0.7716 - val_loss: 0.5916 - val_accuracy: 0.7592
Epoch 

Epoch 172/300
48/48 [==============================] - 1s 16ms/step - loss: 0.4951 - accuracy: 0.8152 - val_loss: 0.5794 - val_accuracy: 0.7690
Epoch 173/300
48/48 [==============================] - 1s 16ms/step - loss: 0.4933 - accuracy: 0.8119 - val_loss: 0.5655 - val_accuracy: 0.7690
Epoch 174/300
48/48 [==============================] - 1s 15ms/step - loss: 0.4983 - accuracy: 0.8102 - val_loss: 0.5782 - val_accuracy: 0.7736
Epoch 175/300
48/48 [==============================] - 1s 16ms/step - loss: 0.4872 - accuracy: 0.8206 - val_loss: 0.5729 - val_accuracy: 0.7729
Epoch 176/300
48/48 [==============================] - 1s 16ms/step - loss: 0.4937 - accuracy: 0.8125 - val_loss: 0.5900 - val_accuracy: 0.7637
Epoch 177/300
48/48 [==============================] - 1s 15ms/step - loss: 0.4879 - accuracy: 0.8135 - val_loss: 0.5663 - val_accuracy: 0.7768
Epoch 178/300
48/48 [==============================] - 1s 16ms/step - loss: 0.4877 - accuracy: 0.8166 - val_loss: 0.5654 - val_accuracy:

Epoch 229/300
48/48 [==============================] - 1s 16ms/step - loss: 0.4330 - accuracy: 0.8440 - val_loss: 0.6126 - val_accuracy: 0.7546
Epoch 230/300
48/48 [==============================] - 1s 16ms/step - loss: 0.4349 - accuracy: 0.8394 - val_loss: 0.5573 - val_accuracy: 0.7886
Epoch 231/300
48/48 [==============================] - 1s 16ms/step - loss: 0.4253 - accuracy: 0.8459 - val_loss: 0.5726 - val_accuracy: 0.7788
Epoch 232/300
48/48 [==============================] - 1s 17ms/step - loss: 0.4217 - accuracy: 0.8463 - val_loss: 0.7480 - val_accuracy: 0.6702
Epoch 233/300
48/48 [==============================] - 1s 16ms/step - loss: 0.4251 - accuracy: 0.8428 - val_loss: 0.5743 - val_accuracy: 0.7723
Epoch 234/300
48/48 [==============================] - 1s 17ms/step - loss: 0.4189 - accuracy: 0.8505 - val_loss: 0.5672 - val_accuracy: 0.7788
Epoch 235/300
48/48 [==============================] - 1s 16ms/step - loss: 0.4595 - accuracy: 0.8255 - val_loss: 0.6529 - val_accuracy:

Epoch 286/300
48/48 [==============================] - 1s 16ms/step - loss: 0.3904 - accuracy: 0.8597 - val_loss: 0.5895 - val_accuracy: 0.7696
Epoch 287/300
48/48 [==============================] - 1s 16ms/step - loss: 0.3375 - accuracy: 0.8877 - val_loss: 0.5550 - val_accuracy: 0.7906
Epoch 288/300
48/48 [==============================] - 1s 16ms/step - loss: 0.3922 - accuracy: 0.8635 - val_loss: 0.5560 - val_accuracy: 0.7925
Epoch 289/300
48/48 [==============================] - 1s 16ms/step - loss: 0.3780 - accuracy: 0.8694 - val_loss: 0.5542 - val_accuracy: 0.7991
Epoch 290/300
48/48 [==============================] - 1s 17ms/step - loss: 0.3335 - accuracy: 0.8934 - val_loss: 0.5594 - val_accuracy: 0.7945
Epoch 291/300
48/48 [==============================] - 1s 16ms/step - loss: 0.4001 - accuracy: 0.8576 - val_loss: 0.8737 - val_accuracy: 0.6060
Epoch 292/300
48/48 [==============================] - 1s 16ms/step - loss: 0.3671 - accuracy: 0.8788 - val_loss: 0.5852 - val_accuracy:

NameError: name 'modelpath' is not defined

In [14]:
predict = model.predict(testX)
predict = np.argmax(predict,axis =1)

    
print(predict)

[0 0 1 ... 2 1 0]


In [15]:
len(predict)

1348

In [16]:

testY = np.argmax(testY,axis =1)
# print(testY)
print(len(comapre_to_test))


1348


In [19]:
Dir = "Wrong-place"
normal_normal = open("./Wrong-place/Normal/Normal.txt","w")
# normal_normal.write("hello")
normal_overexposed = open("./Wrong-place/Normal/OverExposed.txt","w")
normal_UnderExposed = open("./Wrong-place/Normal/UnderExposed.txt","w")
overexposed_normal = open("Wrong-place/OverExposed/Normal.txt","w")
overexposed_overexposed = open("./Wrong-place/OverExposed/OverExposed.txt","w")
overexposed_UnderExposed = open("./Wrong-place/OverExposed/UnderExposed.txt","w")
UnderExposed_normal = open("Wrong-place/UnderExposed/Normal.txt","w")
UnderExposed_overexposed = open("./Wrong-place/UnderExposed/OverExposed.txt","w")
UnderExposed_UnderExposed = open("./Wrong-place/UnderExposed/UnderExposed.txt","w")

In [18]:
for i in range(len(testY)):
    print("%s %s %s"%(testY[i],predict[i],comapre_to_test[i]))
    if testY[i]==0 and predict[i]==0:
        normal_normal.write(comapre_to_test[i]+"\n")
    if testY[i]==0 and predict[i]==1:
        normal_overexposed.write(comapre_to_test[i]+"\n")
    if testY[i]==0 and predict[i]==2:
        normal_UnderExposed.write(comapre_to_test[i]+"\n")
    if testY[i]==1 and predict[i]==0:
        overexposed_normal.write(comapre_to_test[i]+"\n")
    if testY[i]==1 and predict[i]==1:
        overexposed_overexposed.write(comapre_to_test[i]+"\n")
    if testY[i]==1 and predict[i]==2:
        overexposed_UnderExposed.write(comapre_to_test[i]+"\n")
    if testY[i]==2 and predict[i]==0:
        UnderExposed_normal.write(comapre_to_test[i]+"\n")
    if testY[i]==2 and predict[i]==1:
        UnderExposed_overexposed.write(comapre_to_test[i]+"\n")
    if testY[i]==2 and predict[i]==2:
        UnderExposed_overexposed.write(comapre_to_test[i]+"\n")
        
    

1 0 a2959-JI2E3589_P1.5.JPG
0 0 a1426-DSC_0009-2_0.JPG
1 1 a0920-dvf_012_P1.5.JPG
0 2 a3711-LS051025_day_3_LL012_0.JPG
2 2 a2852-JI2E5666_N1.5.JPG
2 2 a1989-MB_070908_016_N1.5.JPG
1 0 a4444-kme_0626_P1.5.JPG
1 0 a2802-IMG_4594_P1.5.JPG
0 2 a0961-WP_Amanda_3rd_bday037_0.JPG
1 0 a3049-20050327_165901__E6B2556_P1.5.JPG
0 0 a2051-LS051026_day_10_LL018_0.JPG
2 2 a2063-Ja_Pe-16_N1.5.JPG
0 0 a2874-IMG_4365_0.JPG
0 0 a0070-IMG_4327_0.JPG
0 0 a2615-IMG_3601_0.JPG
2 2 a1513-050715_224628__I2E4105_N1.5.JPG
0 0 a1264-IMG_2896_0.JPG
0 0 a3306-IMG_4969_0.JPG
2 2 a3267-WP_CRW_0087_N1.5.JPG
0 0 a0436-IMG_2583_0.JPG
0 0 a3245-CRW_0149_0.JPG
2 2 a1081-jmac_MG_6226_N1.5.JPG
0 0 a3899-jmac_DSC2259_0.JPG
2 2 a4498-DSC_0098_N1.5.JPG
2 2 a3430-_I2E5942_N1.5.JPG
0 2 a2924-Ja_Pe-60_0.JPG
0 2 a1438-NKIM_MG_7977_0.JPG
0 0 a2543-kme_590_0.JPG
1 0 a3267-WP_CRW_0087_P1.5.JPG
1 0 a4037-jmac_DSC0807_P1.5.JPG
0 0 a1271-NKIM_MG_3133_0.JPG
1 0 a2845-kme_420_P1.5.JPG
1 1 a3524-kme_205_P1.5.JPG
1 1 a2043-kme_429_P1.5.JPG
